In [1]:
#  Dependencies
import requests
from elasticsearch import Elasticsearch,helpers
import elasticsearch_dsl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import uuid
import random
import json

In [2]:
# host_name = 'https://es-atlas.cern.ch/kibana'
# es-py __version__ is 2.3.0
# es_dsl __version__ is 5.3.0
# initialise elastic search with authorisation

es = Elasticsearch(['es-atlas.cern.ch:9203'],
                                 timeout=10000,
                                 use_ssl=True,
                                 verify_certs=True,
                                 ca_certs='/etc/pki/tls/certs/CERN-bundle.pem',
                                 http_auth='roatlas:la2Dbd5rtn3df!sx')

In [3]:
print(es.ping())
es.info()

True


{'cluster_name': 'atlas',
 'cluster_uuid': 'jGbidQR5T7Ok-8EoeXoFRA',
 'name': 'esatlass02-ites_atlas',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2017-02-09T22:05:32.386Z',
  'build_hash': 'db0d481',
  'build_snapshot': False,
  'lucene_version': '6.4.1',
  'number': '5.2.1'}}

In [4]:
es.cluster.health()

{'active_primary_shards': 2081,
 'active_shards': 4333,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'atlas',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 9,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 14,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [5]:
indices = es.indices.get_aliases().keys()
# len(indices)
rucio= (index for index in indices if('atlas_rucio-events' in index))
events = []
for event in rucio :
    events.append(event)
print('total event indices:',len(events),'\n')
print(events)

total event indices: 34 

['atlas_rucio-events-2017.06.14', 'atlas_rucio-events-2017.06.11', 'atlas_rucio-events-2017.05.28', 'atlas_rucio-events-2017.06.02', 'atlas_rucio-events-2017.06.12', 'atlas_rucio-events-2017.05.30', 'atlas_rucio-events-2017.05.27', 'atlas_rucio-events-2017.05.23', 'atlas_rucio-events-2017.05.21', 'atlas_rucio-events-2017.05.17', 'atlas_rucio-events-2017.05.20', 'atlas_rucio-events-2017.06.07', 'atlas_rucio-events-2017.05.16', 'atlas_rucio-events-2017.06.04', 'atlas_rucio-events-2017.05.15', 'atlas_rucio-events-2017.06.03', 'atlas_rucio-events-2017.06.05', 'atlas_rucio-events-2017.05.19', 'atlas_rucio-events-2017.06.08', 'atlas_rucio-events-2017.06.13', 'atlas_rucio-events-2017.05.18', 'atlas_rucio-events-2017.06.09', 'atlas_rucio-events-2017.06.01', 'atlas_rucio-events-2017.05.22', 'atlas_rucio-events-2017.05.14', 'atlas_rucio-events-2017.06.06', 'atlas_rucio-events-2017.05.31', 'atlas_rucio-events-2017.06.10', 'atlas_rucio-events-test', 'atlas_rucio-events-20

# Total Data Points in the atlas_rucio-events-* index

In [6]:
import time
count=es.count(index='atlas_rucio-events-*')
print('total documents at- {} : {}'.format(time.strftime("%c"), count['count']) )

total documents at- Wed Jun 14 20:02:13 2017 : 371116171


In [7]:
# print(events)
indices_dict = {}
for event in events:
    i = es.count(index=event)
    indices_dict[event] = i['count']
# print('total data points:',sum(int(list(indices_dict.values()))))

print(indices_dict)

{'atlas_rucio-events-2017.06.14': 6365594, 'atlas_rucio-events-2017.05.16': 19012742, 'atlas_rucio-events-2017.06.11': 6900458, 'atlas_rucio-events-2017.05.15': 12680182, 'atlas_rucio-events-2017.05.25': 13829032, 'atlas_rucio-events-2017.06.05': 12903916, 'atlas_rucio-events-2017.06.02': 16490541, 'atlas_rucio-events-2017.06.06': 11178867, 'atlas_rucio-events-2017.05.26': 13379263, 'atlas_rucio-events-2017.06.09': 11647283, 'atlas_rucio-events-2017.06.08': 11591964, 'atlas_rucio-events-2017.05.29': 7871022, 'atlas_rucio-events-2017.05.31': 9625282, 'atlas_rucio-events-2017.05.14': 8569000, 'atlas_rucio-events-2017.06.01': 13624672, 'atlas_rucio-events-2017.05.23': 17165804, 'atlas_rucio-events-2017.06.12': 13273519, 'atlas_rucio-events-2017.05.24': 14482233, 'atlas_rucio-events-2017.06.13': 10110855, 'atlas_rucio-events-2017.05.28': 6314218, 'atlas_rucio-events-2017.06.10': 8682198, 'atlas_rucio-events-2017.06.04': 9845288, 'atlas_rucio-events-2017.06.03': 12449162, 'atlas_rucio-event

In [9]:
indices_dict['atlas_rucio-events-2017.05.25']

13829032

{'atlas_rucio-events-2017.06.02': 16490541, 'atlas_rucio-events-2017.05.22': 12321088, 'atlas_rucio-events-2017.05.13': 8238940, 'atlas_rucio-events-2017.05.26': 13379263, 'atlas_rucio-events-2017.06.08': 11591964, 'atlas_rucio-events-2017.06.03': 12449162, 'atlas_rucio-events-2017.05.19': 14674362, 'atlas_rucio-events-2017.05.28': 6314218, 'atlas_rucio-events-2017.05.27': 8940869, 'atlas_rucio-events-test': 36, 'atlas_rucio-events-2017.05.20': 10152479, 'atlas_rucio-events-2017.05.29': 7871022, 'atlas_rucio-events-2017.06.11': 4003907, 'atlas_rucio-events-2017.06.07': 8533532, 'atlas_rucio-events-2017.05.25': 13829032, 'atlas_rucio-events-2017.05.17': 13075677, 'atlas_rucio-events-2017.06.04': 9845288, 'atlas_rucio-events-2017.06.09': 11647283, 'atlas_rucio-events-2017.05.14': 8569000, 'atlas_rucio-events-2016-06-30': 36737, 'atlas_rucio-events-2017.06.10': 8682198, 'atlas_rucio-events-2017.06.06': 11178867, 'atlas_rucio-events-2017.05.16': 19012742, 'atlas_rucio-events-2017.05.15': 12680182, 'atlas_rucio-events-2017.05.12': 14727794, 'atlas_rucio-events-2017.05.31': 9625282, 'atlas_rucio-events-2017.05.18': 16944116, 'atlas_rucio-events-2017.05.21': 9035848, 'atlas_rucio-events-2017.05.30': 9408605, 'atlas_rucio-events-2017.05.23': 17165804, 'atlas_rucio-events-2017.06.01': 13624672, 'atlas_rucio-events-2017.05.24': 14482233, 'atlas_rucio-events-2017.06.05': 12903916}

In [11]:
stats = es.field_stats(index='atlas_rucio-events-2016-06-30', fields=['payload.created_at'])
#atlas_rucio-events-2016-06-30
print(stats)

{'_shards': {'failed': 0, 'total': 5, 'successful': 5}, 'indices': {'_all': {'fields': {'payload.created_at': {'density': 95, 'max_value': 1467294822000, 'min_value': 1467292656000, 'max_value_as_string': '2016-06-30 13:53:42', 'max_doc': 36737, 'aggregatable': True, 'type': 'date', 'searchable': True, 'sum_doc_freq': 139636, 'sum_total_term_freq': -1, 'min_value_as_string': '2016-06-30 13:17:36', 'doc_count': 34909}}}}}


# Extracting Data from 2-3 Months

In [39]:
# saves data to a dataframe
def extract_data(index, query, scan_size, scan_step):
    resp = es.search(
    index = index,
    scroll = '20m',
    body = query,
    size = scan_step)

    sid = resp['_scroll_id']
    scroll_size = resp['hits']['total']
    results=[]
    for hit in resp['hits']['hits']:
        results.append(hit['_source']['payload'])
    steps = int((scan_size-scan_step)/ scan_step)

    # Start scrolling

    for i in range(steps):
        if i%10==0:
            print("Scrolling index : {} ; step : {} ...\n ".format(index,i))
        resp = es.scroll(scroll_id = sid, scroll = '20m')
        # Update the scroll ID
        sid = resp['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(resp['hits']['hits'])
        if i%10==0:
            print("scroll size: " + str(scroll_size))
        for hit in resp['hits']['hits']:
            results.append(hit['_source']['payload'])
    
    print("\n Done Scrolling through {} !! \n".format(index))
    results = pd.DataFrame(results)
    print(results.info(), '\n')
    return results

In [33]:
index= 'atlas_rucio-events-2017.05.17'
myquery = {
    "query": {
        "term": {
            '_type': 'transfer-done'
            }
        }
    }

# scan_size=indices_dict['atlas_rucio-events-2017.05.17'],
#                     scan_step=10000

data = extract_data(index=index,
                    query=myquery,
                    scan_size=100000,
                    scan_step=1000)

Scrolling index : atlas_rucio-events-2017.05.17 ; step : 0 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 10 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 20 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 30 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 40 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 50 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 60 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 70 ...
 
scroll size: 1000
Scrolling index : atlas_rucio-events-2017.05.17 ; step : 80 ...
 
scroll size: 1000


KeyboardInterrupt: 

In [14]:
data.head()

,account,activity,bytes,checksum-adler,checksum-md5,dst-rse,dst-type,dst-url,duration,guid,...,src-rse,src-type,src-url,started_at,submitted_at,tool-id,transfer-endpoint,transfer-id,transfer-link,transferred_at
0,None,User Subscriptions,36646698,e71f3918,None,TOKYO-LCG2_LOCALGROUPDISK,DISK,srm://lcg-se01.icepp.jp:8446/srm/managerv2?SFN...,214,None,...,PRAGUELCG2_SCRATCHDISK,DISK,srm://golias100.farm.particle.cz:8446/srm/mana...,2017-05-16 23:38:53,2017-05-16 20:57:34,rucio-conveyor,https://fts3-pilot.cern.ch:8446,a5bbd849-41c3-57b1-add7-55e9195ddd3a,https://fts3-pilot.cern.ch:8449/fts3/ftsmon/#/...,2017-05-16 23:42:27
1,None,Production Input,2015682008,e6b85346,None,RO-07-NIPNE_DATADISK,DISK,srm://tbit00.nipne.ro:8446/srm/managerv2?SFN=/...,930,None,...,BNL-OSG2_DATADISK,DISK,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,2017-05-16 23:41:25,2017-05-16 23:41:23,rucio-conveyor,https://fts3-pilot.cern.ch:8446,0c790824-d283-5e59-a693-36a12b51adfd,https://fts3-pilot.cern.ch:8449/fts3/ftsmon/#/...,2017-05-16 23:56:55
2,None,User Subscriptions,36889124,d95f7697,None,TOKYO-LCG2_LOCALGROUPDISK,DISK,srm://lcg-se01.icepp.jp:8446/srm/managerv2?SFN...,20,None,...,FZK-LCG2_SCRATCHDISK,DISK,srm://atlassrm-fzk.gridka.de:8443/srm/managerv...,2017-05-17 00:00:19,2017-05-16 12:09:38,rucio-conveyor,https://fts3-pilot.cern.ch:8446,fb8f25ba-5a09-5a76-a93e-270828fd54f3,https://fts3-pilot.cern.ch:8449/fts3/ftsmon/#/...,2017-05-17 00:00:39
3,None,Production Input,21613485,20e03eb0,None,TRIUMF-LCG2_DATADISK,DISK,srm://srm.triumf.ca:8443/srm/managerv2?SFN=/at...,18,None,...,RAL-LCG2_DATADISK,DISK,srm://srm-atlas.gridpp.rl.ac.uk:8443/srm/manag...,2017-05-17 00:00:11,2017-05-16 06:20:40,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,62ae5abf-ef98-5d3e-9c6e-6f045124d5b2,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2017-05-17 00:00:29
4,None,User Subscriptions,36754943,7942fac2,None,TOKYO-LCG2_LOCALGROUPDISK,DISK,srm://lcg-se01.icepp.jp:8446/srm/managerv2?SFN...,27,None,...,DESY-ZN_SCRATCHDISK,DISK,srm://lcg-se0.ifh.de:8443/srm/managerv2?SFN=/p...,2017-05-16 23:32:20,2017-05-16 20:57:26,rucio-conveyor,https://fts3-pilot.cern.ch:8446,102e4963-7e21-55c2-a246-d56ca6909ac7,https://fts3-pilot.cern.ch:8449/fts3/ftsmon/#/...,2017-05-16 23:32:47


In [34]:
def get_indices_data(indices_list, query, scan_size, scan_step):
    data = pd.DataFrame()
    for index in indices_list:
        tmp_data = extract_data(index, query, scan_size, scan_step)
        data = data.append(tmp_data, ignore_index=True)
#         plot_index_data(tmp_data)
    print(data.info())
#     plot_index_data(dataframe)
    print("\n------------------- ALL DONE !!!!! -----------------------")
    return data

In [35]:
k = [index for index in events if ('atlas_rucio-events-2017.05' in index)]
k

['atlas_rucio-events-2017.05.28',
 'atlas_rucio-events-2017.05.30',
 'atlas_rucio-events-2017.05.27',
 'atlas_rucio-events-2017.05.23',
 'atlas_rucio-events-2017.05.21',
 'atlas_rucio-events-2017.05.17',
 'atlas_rucio-events-2017.05.20',
 'atlas_rucio-events-2017.05.16',
 'atlas_rucio-events-2017.05.15',
 'atlas_rucio-events-2017.05.19',
 'atlas_rucio-events-2017.05.18',
 'atlas_rucio-events-2017.05.22',
 'atlas_rucio-events-2017.05.14',
 'atlas_rucio-events-2017.05.31',
 'atlas_rucio-events-2017.05.26',
 'atlas_rucio-events-2017.05.24',
 'atlas_rucio-events-2017.05.25',
 'atlas_rucio-events-2017.05.29']

In [40]:
df = get_indices_data(indices_list=k, query=myquery, scan_size=30000, scan_step= 5000)

Scrolling index : atlas_rucio-events-2017.05.28 ; step : 0 ...
 
scroll size: 5000

 Done Scrolling through atlas_rucio-events-2017.05.28 !! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 26 columns):
account                0 non-null object
activity               30000 non-null object
bytes                  30000 non-null int64
checksum-adler         30000 non-null object
checksum-md5           0 non-null object
dst-rse                30000 non-null object
dst-type               30000 non-null object
dst-url                30000 non-null object
duration               30000 non-null int64
guid                   0 non-null object
name                   30000 non-null object
previous-request-id    221 non-null object
protocol               30000 non-null object
reason                 30000 non-null object
request-id             30000 non-null object
scope                  30000 non-null object
src-rse                30000 non-null object

In [52]:
z=(df['previous-request-id'][0]) 

In [60]:
# df.to_csv('may.csv')
# id = (df['previous-request-id']!=z)
# id
a=df['previous-request-id'].notnull()

In [62]:
b=df['previous-request-id'][a]
b

47        c48c643a28f3467390801546a39bc836
90        e26c9d992afc439587821ac6d98aad17
296       73c33434d3ce43018eb75906d86cc43a
312       0304ec1b2e9245edaf7e557b80fe2370
351       b235a6c772204d22b4980d639282b54d
356       8d83a76f954a4568813ea0698f3ca43e
734       d2305fb1893547759c9cb4afa844844e
1130      49ec8faf7eb940a69ea0fe314946d61e
1138      4e50b47b67b64debaf8c8d4d62cf1560
1588      db705903b9c74dfc80c41a75b6d75ae7
1652      ee465b085d36427bbf0ac1fbb0a2ac5a
1769      414aa872c8d54672854ac595e54bbabe
2324      76a7be1948514ad3957eae7f50a5ea38
2339      52baaaabfa604f0abd7717b5bb451f07
2765      14950165f19e445b87c59dc2bf72462f
2771      01a12ff105b549c6ae3aeb877651aebe
2867      43142a581ef247f1952746b9c1455ec3
2996      6a5d1232aba04d93b930579c2f1409b7
3182      113d51814b8a4514ac52879c1c7af4f1
3183      8693f725b9634e88901c5bc1a2c2b8a3
3186      36dc5a202ab845d2ba5d032d8d4bcdc0
3190      d6453b8460d54385b2bcc19e9ada9ae0
3200      8c92b7edf59e415bafb41673714c87a0
3215      8

In [16]:
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import MultiMatch, Match

s = Search(using=es, index='atlas_rucio-events-2017.05.17', size=indices_dict['atlas_rucio-events-2017.05.17'])

In [20]:
response = s.execute()
print('Total %d hits found.' % response.hits.total)

Total 13075677 hits found.


In [34]:
# indices_dict['atlas_rucio-events-2017.05.17']
# print((response))
# response.to_dict()
print(response.success())
print(response.took)
print(response.hits.total)
print(len(response.hits.hits))
response.hits.hits[0]

True
193
13075677
10


{'_id': 'AVwVKiRtbU3DyXDECNkW',
 '_index': 'atlas_rucio-events-2017.05.17',
 '_score': 1.0,
 '_source': {'@timestamp': '2017-05-17T06:48:21.353Z',
  '@version': '1',
  'created_at': None,
  'payload': {'account': None,
   'activity': 'Production Input',
   'bytes': 1243350,
   'checksum-adler': 'cae9cfc1',
   'checksum-md5': None,
   'dst-rse': 'FZK-LCG2_DATADISK',
   'dst-type': 'DISK',
   'dst-url': 'srm://atlassrm-fzk.gridka.de:8443/srm/managerv2?SFN=/pnfs/gridka.de/atlas/disk-only/atlasdatadisk/rucio/mc15_13TeV/1f/4f/EVNT.09370789._024906.pool.root.1',
   'duration': 10,
   'guid': None,
   'name': 'EVNT.09370789._024906.pool.root.1',
   'previous-request-id': None,
   'protocol': 'srm',
   'reason': '',
   'request-id': '64b5946f0362400b88b41c48c9deb496',
   'scope': 'mc15_13TeV',
   'src-rse': 'RAL-LCG2_DATADISK',
   'src-type': 'DISK',
   'src-url': 'srm://srm-atlas.gridpp.rl.ac.uk:8443/srm/managerv2?SFN=/castor/ads.rl.ac.uk/prod/atlas/stripInput/atlasdatadisk/rucio/mc15_13TeV/1

In [51]:
for hit in s.scan():
    print(hit._source)

RequestError: TransportError(400, 'illegal_argument_exception', 'No search type for [scan]')

['atlas_rucio-events-2017.05.17',
 'atlas_rucio-events-2017.05.31',
 'atlas_rucio-events-2017.05.20',
 'atlas_rucio-events-2017.05.23',
 'atlas_rucio-events-2017.05.19',
 'atlas_rucio-events-2017.05.21',
 'atlas_rucio-events-2017.05.22',
 'atlas_rucio-events-2017.05.28',
 'atlas_rucio-events-2017.05.18',
 'atlas_rucio-events-2017.05.14',
 'atlas_rucio-events-2017.05.29',
 'atlas_rucio-events-2017.05.25',
 'atlas_rucio-events-2017.05.30',
 'atlas_rucio-events-2017.05.15',
 'atlas_rucio-events-2017.05.24',
 'atlas_rucio-events-2017.05.26',
 'atlas_rucio-events-2017.05.27',
 'atlas_rucio-events-2017.05.16']

In [11]:
# Deprecated code - wprks with ES-P\PY 2.1

# # # implementing scan and scroll
# def extract_data(index, scan_size, scan_step):
    
#     # Initialize the scroll
#     page = es.search(
#     index = index,
#     scroll = '20m',
#     search_type = 'scan',
#     size = scan_step
#     )
#     sid = page['_scroll_id']
#     scroll_size = page['hits']['total']
#     results=[]
#     steps = int(scan_size / scan_step)

#     # Start scrolling

#     for i in range(steps):
#         print("Scrolling ",index, "...\n")
#         page = es.scroll(scroll_id = sid, scroll = '20m')
#         # Update the scroll ID
#         sid = page['_scroll_id']
#         # Get the number of results that we returned in the last scroll
#         scroll_size = len(page['hits']['hits'])
#         print("scroll size: " + str(scroll_size))
#         for hit in page['hits']['hits']:
#             results.append(hit['_source'])
        
#     results = pd.DataFrame(results)
#     return results

# a = extract_data(index='atlas_rucio-events-2017.05.25', 
#                  scan_size=indices_dict['atlas_rucio-events-2017.05.25'],
#                  scan_step = 100)

# # a=helpers.scan(es, index='atla-rucio-events-*')

RequestError: TransportError(400, 'illegal_argument_exception', 'No search type for [scan]')

In [17]:

# query = {
#     "query": {
#         "match" : {
#             "_type" : "rucio-event"
#         },
#         "wildcard" : {
#             "payload" : {
#                 "created_at" : "2016-06-30*"
#             }
#         }

#     },
#     "size": 10
# }

# query = {
#     "query": {
#         "match" : {
#             "_type" : "rucio-event"
#         }

#     },
#     "sort": [
#         { "created_at": {"order":"asc"}}
#     ]
# }

res = es.search(index='atlas_rucio-events-2017.06.12', size=10)
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVyaTMsjShn3HZ18OIdw',
    '_index': 'atlas_rucio-events-2017.06.12',
    '_score': 1.0,
    '_source': {'@timestamp': '2017-06-12T03:15:42.254Z',
     '@version': '1',
     'created_at': '2017-06-12 03:15:33',
     'payload': {'bytes': 196702,
      'duration': 0.00820302963256836,
      'name': 'panda.um.user.ekay.DATA16.00310249.20170414.195021.log.11178487.000579.log.tgz',
      'rse': 'CERN-PROD_SCRATCHDISK',
      'scope': 'panda',
      'url': 'root://eosatlas.cern.ch:1094//eos/atlas/atlasscratchdisk/rucio/panda/ed/bc/panda.um.user.ekay.DATA16.00310249.20170414.195021.log.11178487.000579.log.tgz'},
     'tags': ['ites'],
     'type': 'deletion-done'},
    '_type': 'deletion-done'},
   {'_id': 'AVyaTMsjShn3HZ18OId3',
    '_index': 'atlas_rucio-events-2017.06.12',
    '_score': 1.0,
    '_source': {'@timestamp': '2017-06-12T03:15:42.257Z',
     '@version': '1',
     'created_at': '2017-06-12 03:15:

In [36]:
ev = (res['hits']['hits'][0])

type(ev['_source']['@timestamp'])

str

In [ ]:
myquery={
      'query': {
        'bool': {
          'must': {
            'match': {'description': 'fix'}
          },
          'must_not': {
            'term': {'files': 'test_elasticsearch'}
          }
        }
      }
    }

In [6]:
# es.get_source(index='atlas_rucio-events-*', doc_type=, id= )     , filter_path='hits.hits._source'
es.search(index='atlas_rucio-events-*', size=5)

{'_shards': {'failed': 0, 'successful': 160, 'total': 160},
 'hits': {'hits': [{'_id': 'AVWhdatvHmRcx6WJRPGB',
    '_index': 'atlas_rucio-events-2016-06-30',
    '_score': 1.0,
    '_source': {'@timestamp': '2016-06-30T13:18:04.231Z',
     '@version': '1',
     'event_type': 'erase',
     'payload': {'account': 'root',
      'created_at': '2016-06-30 13:17:51',
      'name': 'panda.um.data16_13TeV.00302265.physics_Main.merge.DAOD_JETM10.f709_m1620_p2689_tid08833658_00.85258329',
      'scope': 'panda'},
     'type': 'rucio-event'},
    '_type': 'rucio-event'},
   {'_id': 'AVWhdatvHmRcx6WJRPGF',
    '_index': 'atlas_rucio-events-2016-06-30',
    '_score': 1.0,
    '_source': {'@timestamp': '2016-06-30T13:18:04.231Z',
     '@version': '1',
     'event_type': 'deletion-planned',
     'payload': {'bytes': 67238,
      'created_at': '2016-06-30 13:17:52',
      'file-size': 67238,
      'name': 'user.bertella.014491._1478363636.log.tgz',
      'rse': 'IN2P3-CPPM_LOCALGROUPDISK',
      'scop

In [14]:
myquery = {"query": {"match_all": {}}}

In [29]:
next(a)

RequestError: TransportError(400, 'illegal_argument_exception', 'No search type for [scan]')

In [20]:
rucio_data = extract_data(index='atlas_rucio-events-*', scan_size = 1000, scan_step=100)

RequestError: TransportError(400, 'illegal_argument_exception', 'No search type for [scan]')

# atlas_rucio-events-*

## event format -

```json
{
    "_index": "atlas_rucio-events-2017.06.05",
    "_type": "deletion-done",
    "_id": "AVx2vViw_PfPfg29wVZQ",
    "_score": null,
    "_source": {
        "created_at": "2017-06-05 05:32:12",
        "payload": {
            "name": "user.croland.11304844.EXT0._000704.events.lhe.copy",
            "url": "https://webdav.mwt2.org:2881/atlasscratchdisk/rucio/user/croland/26/05/user.croland.11304844.EXT0._000704.events.lhe.copy",
            "bytes": 6575271,
            "rse": "MWT2_UC_SCRATCHDISK",
            "duration": 0.20047616958618164,
            "scope": "user.croland"
        },
        "@version": "1",
        "@timestamp": "2017-06-05T05:32:18.604Z",
        "type": "deletion-done",
        "tags": [
            "ites"
        ]
    },
    "fields": {
        "@timestamp": [
            1496640738604
        ]
    },
    "sort": [
        1496640738604
    ]
}
```